<a href="https://colab.research.google.com/github/scottthomaswx/Great-Lakes-WW3/blob/main/GreatLakesWaveWatchIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# @title
#Load and import packages
%%capture
!pip install numpy
!pip install xarray[complete]
!pip install metpy
!pip install matplotlib
!pip install cartopy
!pip install datetime
!pip install requests
!pip install pyproj
!pip install pandas
!pip install cfgrib
!apt-get install -y libeccodes0
!pip install cfgrib xarray
!pip install sys
!pip install warnings
import xarray as xr
import numpy as np
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from datetime import datetime, timezone
import requests
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from metpy.plots import add_timestamp
import pyproj as proj
import pandas as pd
import sys
import warnings
import matplotlib.animation as animation
warnings.filterwarnings('ignore')
!rm -rf sample_data
!mkdir WW3_Img

##Set Relevant Parameters

In [71]:
data_source = 'Operational' # @param ["GLERL Experimental", "Operational"]
extent = 'Lake Huron' # @param ["Lakes Michigan and Huron", "Lake Erie", "Lake Ontario", "Lake Superior", "Pictured Rocks Natl Lakeshore", "Lake Michigan", "Lake Huron"]

#Obtain Wavewatch Three Data

In [24]:
# @title
#Get and Save WaveWatch III Data

if data_source == 'GLERL Experimental':
  WW = 'https://www.glerl.noaa.gov/emf/waves/WW3/images/nomads.grib2'
elif data_source == 'Operational':
  now_utc = datetime.now(timezone.utc)
  run_12z =  datetime.now(timezone.utc).replace(hour=13, minute = 27, second=0)
  run_18z =  datetime.now(timezone.utc).replace(hour=19, minute=27, second=0)
  run_00z =  datetime.now(timezone.utc).replace(hour=1, minute=27, second=0)
  run_06z = datetime.now(timezone.utc).replace(hour=7, minute=27, second=0)
  if now_utc > run_12z and now_utc < run_18z:
    model_year = str(datetime.now(timezone.utc).year)
    if datetime.now(timezone.utc).month < 10:
      model_month = '0'+str(datetime.now(timezone.utc).month)
    else:
      model_month = str(datetime.now(timezone.utc).month)
    model_day = str(datetime.now(timezone.utc).day)
    WW = f'https://nomads.ncep.noaa.gov/cgi-bin/filter_glwu.pl?dir=%2Fglwu.{model_year}{model_month}{model_day}&file=glwu.grlc_2p5km.t13z.grib2&all_var=on&all_lev=on'
  elif now_utc > run_18z and now_utc < run_00z:
    model_year = str(datetime.now(timezone.utc).year)
    if datetime.now(timezone.utc).month < 10:
          model_month = '0'+str(datetime.now(timezone.utc).month)
    else:
        model_month = str(datetime.now(timezone.utc).month)
    model_day = str(datetime.now(timezone.utc).day)
    WW = f'https://nomads.ncep.noaa.gov/cgi-bin/filter_glwu.pl?dir=%2Fglwu.{model_year}{model_month}{model_day}&file=glwu.grlc_2p5km.t19z.grib2&all_var=on&all_lev=on'
  elif now_utc > run_00z and now_utc < run_06z:
    model_year = str(datetime.now(timezone.utc).year)
    if datetime.now(timezone.utc).month < 10:
          model_month = '0'+str(datetime.now(timezone.utc).month)
    else:
          model_month = str(datetime.now(timezone.utc).month)
    model_day = str(datetime.now(timezone.utc).day)
    WW = f'https://nomads.ncep.noaa.gov/cgi-bin/filter_glwu.pl?dir=%2Fglwu.{model_year}{model_month}{model_day}&file=glwu.grlc_2p5km.t01z.grib2&all_var=on&all_lev=on'
  elif now_utc > run_06z and now_utc < run_12z:
    model_year = str(datetime.now(timezone.utc).year)
    if datetime.now(timezone.utc).month < 10:
        model_month = '0'+str(datetime.now(timezone.utc).month)
    else:
        model_month = str(datetime.now(timezone.utc).month)
  model_day = str(datetime.now(timezone.utc).day)
  WW = f'https://nomads.ncep.noaa.gov/cgi-bin/filter_glwu.pl?dir=%2Fglwu.{model_year}{model_month}{model_day}&file=glwu.grlc_2p5km.t07z.grib2&all_var=on&all_lev=on'
r1 = requests.get(WW)
file1='/content/latestwavewatch.grib2'
with open(file1, 'wb') as f1:
               f1.write(r1.content)

In [26]:
# @title
%%capture
ww3 = xr.open_dataset(file1, engine="cfgrib")
swh = ww3.metpy.parse_cf('swh').metpy.convert_units('ft')

#Map Wavewatch III Data
## Output Will Be In Folder In Left Pane


In [ ]:
# @title
%%capture
float_formatter = "{:03d}".format
i=0
rounded = np.array(swh.time, dtype='datetime64[m]')
while i <  len(ww3['valid_time']):
    #Create Figure and Subplot
    fig = plt.figure(figsize=(14,8), facecolor='white', edgecolor = 'black', linewidth = 6)

    ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
    ax.set_box_aspect(0.7)

    #Create Wave Contours
    wave_fill = ax.contourf(swh.longitude,swh.latitude,swh.isel(step=i), cmap='magma', levels = np.arange(0,21,1), zorder=7, transform=ccrs.PlateCarree())
    wave_cont = ax.contour(swh.longitude,swh.latitude,swh.isel(step=i), color='black', levels = np.arange(0,21,1), zorder=7, transform=ccrs.PlateCarree())
    wavelabels= ax.clabel(wave_cont, fmt='%01d', fontsize = 'large', colors = 'black')

    #Plot and SUbplot Settings
    plt.colorbar(wave_fill, orientation='vertical',pad=0.02, aspect=50, shrink=0.935, label = 'Wave Height (ft)')
    if data_source == 'GLERL Experimental':
      plt.figtext(0.125, 0.08, f'SCOTT THOMAS | SOURCE: GLERL Experimental WW3', weight='bold', color='black', fontsize=16, ha='left', alpha=0.8)
    elif data_source == 'Operational':
      plt.figtext(0.125, 0.08, f'SCOTT THOMAS | SOURCE: NCEP Operational WW3', weight='bold', color='black', fontsize=16, ha='left', alpha=0.8)
    plt.figtext(0.125, 0.04, f'Note: Areas in White Either Have Missing Data OR Are Ice Covered', color='black', fontsize=12, ha='left', alpha=0.8)
    plt.figtext(0.125, 0.895, f'Runtime: {rounded} at FHR: {i}', ha='left', weight='bold', fontsize=12)
    ax.coastlines(color='black',resolution='10m',zorder=99)
    ax.add_feature(cfeature.STATES.with_scale('50m'),edgecolor='black', zorder=9, facecolor = 'lightgray')


    #Adjust Lat/Long To Map
    if extent == 'Lakes Michigan and Huron':
      ax.set_extent((-88.25,-81,41.5,46.25))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lakes Michigan and Huron', ha='left', weight='bold', fontsize=15)
    elif extent == 'Lake Superior':
      ax.set_extent((-92.25,-84.25,46,49))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lake Superior', ha='left', weight='bold', fontsize=15)
    elif extent == 'Pictured Rocks Natl Lakeshore':
      ax.set_extent((-87.25,-85.75,46.25, 47))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Pictured Rocks Natl. Lakeshore', ha='left', weight='bold', fontsize=15)
    elif extent == 'Lake Ontario':
      ax.set_extent((-80,-76,42.25, 44.25))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lake Ontario', ha='left', weight='bold', fontsize=15)
    elif extent == 'Lake Erie':
      ax.set_extent((-83.5, -78.75, 41.25, 43.25))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lake Erie', ha='left', weight='bold', fontsize=15)
    elif extent == 'Lake Michigan':
      ax.set_extent((-88.25,-85.75,41.5, 46))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lake Michigan', ha='left', weight='bold', fontsize=15)
    elif extent == 'Lake Huron':
      ax.set_extent((-85,-79.75,42.75, 46.75))
      plt.figtext(0.125, 0.92, f'WW3 Significant Wave Height (Feet) - Lake Huron', ha='left', weight='bold', fontsize=15)
    add_timestamp(ax, swh['valid_time'][i].values.astype('datetime64[ms]').astype('O'),
                  y=0.02, high_contrast=True, color = 'white', pretext = 'Valid: ', zorder=10)
    #Save Figure
    plt.savefig('/content/WW3_Img/WW3_FHR'+str(float_formatter(i))+'.png', dpi=100, bbox_inches='tight')
    i+=1